In [1]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split

def prep_tables():
    employees = pd.read_csv('../data/turnover.csv')
    
    employees.insert(0, 'emp_ID', range(1, len(employees)+1))
    # holdout 1000 observation for bulk-loading and making prediction
    # this heldout data will simulate in-production data
    train, test = train_test_split(employees,
                                   test_size=1000,
                                   random_state=12345)
    
    test = test.reset_index(drop=True)
    
    # drop the Y variable for the test data, so that it can play
    # the role of in-production data
    # this data will be used for bulk-load predictions
    jul17, jan18 = train_test_split(test,
                                    test_size=500,
                                    random_state=12345)
    jul17 = jul17.reset_index(drop=True)
    jan18 = jan18.reset_index(drop=True)
    return (train, jul17, jan18)

In [2]:
train, test, test2 = prep_tables()

In [5]:
print(list(train))
print(list(test))

['emp_ID', 'satisfaction_level', 'last_evaluation', 'number_project', 'average_montly_hours', 'time_spend_company', 'Work_accident', 'left', 'promotion_last_5years', 'sales', 'salary']
['emp_ID', 'satisfaction_level', 'last_evaluation', 'number_project', 'average_montly_hours', 'time_spend_company', 'Work_accident', 'left', 'promotion_last_5years', 'sales', 'salary']


In [3]:
train.to_csv('../data/train.csv', index = False)

In [4]:
test2.to_csv('../data/test.csv', index = False)

## Old

In [ ]:
import config
from sqlalchemy import create_engine
import pandas as pd
import os


def read_data(table_name):
    """Read the data from a specified table in the database.

    This function can be used to query both the train set (used to fit model)
    and the "in-production" data (used to make bulk predictions).
    This function will always return a dataframe with all columns stored
    in the table that it is asked to query.

    Args:
        table_name (str): Name of table to be queried

    Returns:
        pandas.DataFrame: A dataframe with the data read from database.
    """
    engine = create_engine(config.database_config)
    sql = "select * from " + table_name
    data = pd.read_sql_query(sql, con=engine)
    return data